In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time
from scipy import signal
from time import sleep
import sys
import meep as mp

In [2]:
M=2**21
tsim=7*10**(-12)
eps0 = 8.854187e-12
hbar = 1.05457182e-34
c = 2.99792458e8
kb = 1.380649e-23
hdk=7.63823258e-12
deltap = np.sqrt(1/3/eps0/hbar/c)*kb
T=50000

In [13]:
#define square root of Dn function
def Dnsqt(wla,T):
    return np.sqrt(6*hdk**2*wla/(np.exp(hdk*wla/T)-1)/T**2/np.pi)

#generate an array of the square root of Dn function
Dsqt = []
for n in range(1,M):
    Dsqt.append(Dnsqt(2*np.pi*n/tsim,T))

#define how many times to run to average the results
Ncomp=100


#define the starting sum of the result
Efsum = np.zeros(2*M)

#generate random numbers
M0 = np.random.normal(0,1)
Mlp = np.random.normal(0,1,M-1)
Nlp = np.random.normal(0,1,M-1)
MM = np.random.normal(0,1)
    
    #generate the array of the fourier transform of E field, we generate l=0 and l=-M separately. 
    #for l=1 to M-1 and l=-1 to -(M-1) are conjutated to each other. 
    #we contruct the whole array by gluing them together by the order from l=0 -> l=M-1 -> l=-M -> l=-(m-1) -> l=-1
Ef0 = np.array([M0*np.sqrt(6*hdk/np.pi/T)])
Eflp = np.multiply((Mlp + 1j*Nlp), Dsqt)
Eflm = np.flip(np.conjugate(Eflp))
EfM = np.array([MM*Dnsqt(2*np.pi*M/tsim,T)])
Ef = np.hstack((Ef0,Eflp,EfM,Eflm))
    
    #we normalize Ef and compute its inverse fast fourier transform
Efn = deltap*T/np.sqrt(tsim)*Ef
En = np.fft.ifft(Efn)

S_Efn = lambda t : Efn[t]
sources = [
    mp.Source(src=mp.CustomSource(src_func=S_Efn,
                                  start_time = 0,
                                  end_time = 1000),
        center=mp.Vector3(1),
        size=mp.Vector3(1),
        component=mp.Ez,
    )
]


In [16]:
eps = 13  # dielectric constant of waveguide
w = 1.2  # width of waveguide
r = 0.36  # radius of holes
d = 1.4  # defect spacing (ordinary spacing = 1)
N = 3  # number of holes on either side of defect

resolution = 20  # pixels/μm
sy = 6  # size of cell in y direction (perpendicular to wvg.)
pad = 2  # padding between last hole and PML edge
dpml = 1  # PML thickness
sx = 2 * (pad + dpml + N) + d - 1  # size of cell in x direction
cell = mp.Vector3(sx,)
pml_layers = mp.PML(dpml)

geometry = [
    mp.Block(
        center=mp.Vector3(),
        size=mp.Vector3(w),
        material=mp.Medium(epsilon=eps),
    )
]
sim = mp.Simulation(
    cell_size=cell,
    geometry=geometry,
    sources=[sources],
    boundary_layers=[pml_layers],
    resolution=resolution,
)


In [18]:
sim.run(
    until_after_sources=mp.stop_when_fields_decayed(
        50, mp.Hz, mp.Vector3(0.12, -0.37), 1e-8
    )
)

-----------
Initializing structure...
time for choose_chunkdivision = 8.41618e-05 s
Working in 2D dimensions.
Computational cell is 12.4 x 0.05 x 0 with resolution 20
     block, center = (0,0,0)
          size (1.2,0,0)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (13,13,13)
time for set_epsilon = 0.0021379 s
-----------


AttributeError: 'list' object has no attribute 'center'